** Neo4j
Graph Creation:
* Build a graph database to represent a social network, with nodes for users and relationships for friendships.
* Create nodes for users and relationships to model their interactions.

In [56]:
from neo4j import GraphDatabase

In [57]:
uri = 'neo4j+s://ff864c08.databases.neo4j.io'
username = 'neo4j'
password = 'JtjMFkHtu1L2YYJ-jDAuNCMhfyjzC8JJTzycaqp1ZlE'
driver = GraphDatabase.driver(uri, auth=(username, password))

In [58]:
def create_node(transaction,name):
    transaction.run("CREATE (:Person {name:$name})",name=name)

with driver.session() as session:
    session.execute_write(create_node,'Dovile')
    session.execute_write(create_node,'Jonas')
    session.execute_write(create_node,'Ana')
    session.execute_write(create_node,'Petras')
    session.execute_write(create_node,'Ada')
    session.execute_write(create_node,'Vytas')

In [59]:
def create_friendship(transaction, name1, name2):
  transaction.run(
      "MATCH (person1:Person {name: $name1}), (person2:Person {name: $name2})"
      "CREATE (person1)-[:FRIEND]->(person2)",
      name1=name1,
      name2=name2)


In [89]:
with driver.session() as session:
    session.execute_write(create_friendship,'Jonas','Dovile')
    session.execute_write(create_friendship,'Dovile','Ana')
    session.execute_write(create_friendship,'Petras','Jonas')
    session.execute_write(create_friendship,'Petras','Ana')
    session.execute_write(create_friendship,'Ada','Ana')
    session.execute_write(create_friendship,'Petras','Vytas')
    session.execute_write(create_friendship,'Petras','Ada')

Querying Graph Data:
* Write a Cypher query to find all friends of a given user up to a certain depth (degree of separation).
* Retrieve common friends between two users in the graph.

In [7]:
def friends_of_user(session, user_name, depth):
    session = driver.session()
    query = ("MATCH (user:Person {name: $name})"
        "WITH user "
        "MATCH (user)-[:FRIEND*1.." + str(depth) + "]->(friend) "
        "WHERE friend.name IS NOT NULL "
        "RETURN friend.name")

    results = session.run(query, name=user_name)
    return [result['friend.name'] for result in results]

friends_of_dovile = friends_of_user(session, 'Dovile', 2)
print('Friends of Dovile:', friends_of_dovile)


Friends of Dovile: ['Ana', 'Jonas']


In [8]:
def common_friends_between_users(session, user1_name, user2_name):
    session = driver.session()
    query = ("MATCH (user1:Person {name: $name1}), (user2:Person {name: $name2}) "
        "WITH user1, user2 "
        "MATCH (user1)-[:FRIEND]->(commonFriend)<-[:FRIEND]-(user2) "
        "WHERE commonFriend.name IS NOT NULL "
        "RETURN commonFriend.name")

    results = session.run(query, name1=user1_name, name2=user2_name)
    return [result['commonFriend.name'] for result in results]

common_friends_of_dovile_and_petras = common_friends_between_users(session, 'Dovile', 'Petras')
print('Common friends of Dovile and Jonas:', common_friends_of_dovile_and_petras)

Common friends of Dovile and Jonas: ['Ana', 'Jonas']


Recommendation System:
* Implement a basic recommendation system by finding users who have similar interests based on their interactions.
* Propose movies, books, or other items that a user might be interested in based on their connections' preferences.

In [75]:
def add_hobbies_to_person(transaction, name, hobbies):
    query = "MATCH (p:Person {name:$name}) SET p.hobbies = $hobbies"
    transaction.run(query, name=name, hobbies=hobbies)

with driver.session() as session:
    persons = {'Dovile': ['Reading', 'Swimming'],
                'Jonas': ['Cooking', 'Hiking'],
                'Ana': ['Painting', 'Dancing'],
                'Petras': ['Photography', 'Cycling'],
                'Ada': ['Gardening', 'Yoga'],
                'Vytas': ['Traveling', 'Playing Guitar']}

    for name, hobbies in persons.items():
        session.execute_write(add_hobbies_to_person, name, hobbies)


In [74]:
def get_suggested_hobbies(session, person_name):
    query = "MATCH (:Person {name: $name})-[:FRIEND]->(friend:Person) "
    "RETURN COLLECT(DISTINCT friend.hobbies) AS suggested_hobbies"
    result = session.run(query, name=person_name)
    record = result.single()
    if record:
        suggested_hobbies = [hobby for hobbies in record['suggested_hobbies'] for hobby in hobbies]
        return suggested_hobbies
    return []

with driver.session() as session:
    person_name = 'Dovile'
    suggested_hobbies = get_suggested_hobbies(session, person_name)
    print(f'Suggested hobbies for {person_name}: {suggested_hobbies}')


Suggested hobbies for Dovile: ['Cycling', 'Yoga']


Pathfinding:
* Use graph algorithms in Neo4j to find the shortest path between two nodes in the graph.
* Find the path of mutual friends between two users.

In [96]:
def find_shortest_path(transaction, name1, name2):
    result = transaction.run("MATCH (person1:Person {name: $name1}), (person2:Person {name: $name2}), "
                             "path = SHORTESTPATH((person1)-[*]->(person2)) "
                             "RETURN nodes(path), relationships(path)", name1=name1, name2=name2)
    return [{'nodes': record['nodes(path)'], 'relationships': record['relationships(path)']} for record in result]

with driver.session() as session:
    results = session.read_transaction(find_shortest_path, 'Dovile', 'Ana')
    if results:
        nodes = results[0]['nodes']
        relationships = results[0]['relationships']
        
        print('Shortest path nodes:')
        for node in nodes:
            print(f'- {node["name"]}')
        
        print('Relationships:')
        for relationship in relationships:
            print(f'- {relationship.type}')
    else:
        print('No path found between the two persons')



Shortest path nodes:
- Dovile
- Ana
Relationships:
- FRIEND


C:\Users\dovil\AppData\Local\Temp\ipykernel_22984\2873084052.py:8: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(find_shortest_path, 'Dovile', 'Ana')


In [113]:
def find_mutual_friends_path(transaction, name1, name2):
    query = ("MATCH (p1:Person {name: $name1})-[:FRIEND]->(mf:Person)-[:FRIEND]->(p2:Person {name: $name2}) "
    "RETURN COLLECT(DISTINCT mf) AS mutualFriends")
    
    result = transaction.run(query, name1=name1, name2=name2)
    record = result.single()
    if record:
        mutual_friends = record["mutualFriends"]
        return [node["name"] for node in mutual_friends]
    return []

with driver.session() as session:
    user1 = 'Petras'
    user2 = 'Dovile'
    mutual_friends_path = session.read_transaction(find_mutual_friends_path, user1, user2)
    
    if mutual_friends_path:
        print(f'Path of mutual friends between {user1} and {user2}:')
        for friend in mutual_friends_path:
            print(f' {friend}')
    else:
        print('No path of mutual friends found between the two users')


Path of mutual friends between Petras and Dovile:
 Jonas


C:\Users\dovil\AppData\Local\Temp\ipykernel_22984\695707109.py:15: DeprecationWarning: read_transaction has been renamed to execute_read
  mutual_friends_path = session.read_transaction(find_mutual_friends_path, user1, user2)
